# **4.28_clean_messy_data**

In [1]:
import pandas as pd

整洁数据，根据埃德加科德的第三范式，包括以下三个特点：

   1)每列是一个变量

   2)每行是一个观察值

   3)每个单元格是一个值

   任何不符合以上三个特点的数据都是乱数据。

### 一、对数据进行转置

当数据的列和行是反的，也就是每列是观察值，每行是变量，需要对数据进行转置

DataFrame的T属性，能返回一个行和列调换后的DataFrame

In [2]:
df1 = pd.read_csv('example4.csv', index_col=0)
df1

,0,1,2
姓名,张三,李四,王五
年龄,30,25,28
地址,北京,上海,广州


In [3]:
df1 = df1.T
df1

,姓名,年龄,地址
0,张三,30,北京
1,李四,25,上海
2,王五,28,广州


### 二、对列进行拆分

当一列有多个变量时，把一列拆分成多列

In [4]:
df2 = pd.read_csv("example5.csv")
df2

,城市,人口密度
0,城市A,28.53万人/0.87平方公里
1,城市B,37.83万人/2.19平方公里
2,城市C,15.3万人/1.57平方公里


#### (一)、拆分列

1. 针对Python字符串

   split方法，参数可以指定分隔符，就会得到拆分后字符串所组成的列表

In [5]:
"28.53万人/0.87平方公里".split("/")

['28.53万人', '0.87平方公里']

2. 针对Series

   str.split方法，参数可以指定分隔符，拆分字符串，调用后返回一个新的Series，每一个元素是一个列表。

    但这没有达到拆分列的效果，因为Series只能表示DataFrame的一列，而我们希望拆分成多列。

In [6]:
df2['人口密度'].str.split('/')

0    [28.53万人, 0.87平方公里]
1    [37.83万人, 2.19平方公里]
2     [15.3万人, 1.57平方公里]
Name: 人口密度, dtype: object

3. 拆分列(其实也是针对Series)

   str.split方法，额外指定一个可选参数expand=True，表示把分隔后的结果分别用单独的Series表示，这样输出结果就会变成一个包含多列的DataFrame

In [7]:
df2['人口密度'].str.split('/', expand=True)

,0,1
0,28.53万人,0.87平方公里
1,37.83万人,2.19平方公里
2,15.3万人,1.57平方公里


#### (二)、把新生成的DataFrame添加进原先的DataFrame

其实就是添加多列

在DataFrame后面方括号里放入列表，列表里放入多个列名，赋值给要加入的新列所组成的DataFrame

In [8]:
df2[['人口', '面积']] = df2['人口密度'].str.split('/', expand=True)
df2

,城市,人口密度,人口,面积
0,城市A,28.53万人/0.87平方公里,28.53万人,0.87平方公里
1,城市B,37.83万人/2.19平方公里,37.83万人,2.19平方公里
2,城市C,15.3万人/1.57平方公里,15.3万人,1.57平方公里


#### (三)、删除拆分前的列

drop方法，指定axis=1

In [9]:
df2 = df2.drop('人口密度', axis=1)
df2

,城市,人口,面积
0,城市A,28.53万人,0.87平方公里
1,城市B,37.83万人,2.19平方公里
2,城市C,15.3万人,1.57平方公里


### 三、把不同列合并成一列

In [10]:
df3 = pd.DataFrame({'姓': ['张', '李', '王'], '名': ['三', '四', '五'], '年龄': [30, 25, 35]})
df3

,姓,名,年龄
0,张,三,30
1,李,四,25
2,王,五,35


1. 拼接字符串

    str.cat方法，参数传入拼接的Series，可以拼接2个Series。

    可以传入可选参数sep，来指定拼接时的分隔符

2. 添加列

3. 删除拼接前的列

In [11]:
df3.姓.str.cat(df3.名)

0    张三
1    李四
2    王五
Name: 姓, dtype: object

In [12]:
df3['姓'].str.cat(df3['名'], sep='-')

0    张-三
1    李-四
2    王-五
Name: 姓, dtype: object

In [13]:
df3['姓名'] = df3.姓.str.cat(df3.名)
df3 = df3.drop(['姓', '名'], axis=1)
df3

,年龄,姓名
0,30,张三
1,25,李四
2,35,王五


### 四、把宽数据转换成长数据

In [14]:
df4 = pd.read_csv('example6.csv')
df4

,国家代码,年份,男性年龄组（0-4岁）,男性年龄组（5-14岁）,男性年龄组（15-24岁）,女性年龄组（0-4岁）,女性年龄组（5-14岁）,女性年龄组（15-24岁）
0,CN,2019,100,200,500,80,150,400
1,US,2019,50,150,300,40,100,200
2,JP,2019,30,120,250,20,80,150
3,IN,2019,80,180,400,60,120,300


即重塑列，具体来说是在把一部分列名的值，转换为变量的值

pd.melt方法，可以帮我们进行这方面的转换。

第一个参数传入要转换的DataFrame

可选参数id_vars传入一个列表，里面放入想保持原样的列，那么除了id_vars列表里面之外的列，都会被视为要进行转换的列

可选参数var_name，要被赋值为在转换后的DataFrame中，包含原本列名值的新列列名

可选参数value_name，要被赋值为在转换后的DataFrame中，包含原本变量值的新列列名

运行后，就会返回一个从原本宽数据转换成长数据的DataFrame

In [15]:
df4 = pd.melt(df4, 
             id_vars=['国家代码', '年份'], 
             var_name='年龄组', 
             value_name='肺结核病例数')
df4

,国家代码,年份,年龄组,肺结核病例数
0,CN,2019,男性年龄组（0-4岁）,100
1,US,2019,男性年龄组（0-4岁）,50
2,JP,2019,男性年龄组（0-4岁）,30
3,IN,2019,男性年龄组（0-4岁）,80
4,CN,2019,男性年龄组（5-14岁）,200
5,US,2019,男性年龄组（5-14岁）,150
6,JP,2019,男性年龄组（5-14岁）,120
7,IN,2019,男性年龄组（5-14岁）,180
8,CN,2019,男性年龄组（15-24岁）,500
9,US,2019,男性年龄组（15-24岁）,300


### 五、对行进行拆分

In [16]:
df5 = pd.read_csv("example7.csv")
df5

,学生姓名,学号,课程列表
0,张三,1,"['数学', '物理']"
1,李四,2,"['英语', '化学', '历史']"
2,王五,3,"['语文', '数学', '英语', '政治']"
3,赵六,4,"['物理', '生物']"


In [17]:
# 虽然上面的课程列表变量看起来像是列表，但实际上是字符串
# 比如第一行里的课程列表实际为字符串"['数学', '物理']"，而不是列表['数学', '物理']
# 所以我们需要先将字符串形式的列表转换为实际的列表对象，这样explode才能对列表进行拆分
import json
df5['课程列表'] = df5['课程列表'].apply(lambda x: json.loads(x.replace("'", "\"")))
df5

,学生姓名,学号,课程列表
0,张三,1,"[数学, 物理]"
1,李四,2,"[英语, 化学, 历史]"
2,王五,3,"[语文, 数学, 英语, 政治]"
3,赵六,4,"[物理, 生物]"


当某列的值都是列表，而不是一个个独立的值

df.explode方法，传入需要拆分的变量名，就会返回一个把该列列表中每一个元素，转换为单独一行的新DataFrame

这个方法很实用，当每个单元格是一个值而不是列表时，更好进行统计

In [18]:
df5 = df5.explode('课程列表')
df5

,学生姓名,学号,课程列表
0,张三,1,数学
0,张三,1,物理
1,李四,2,英语
1,李四,2,化学
1,李四,2,历史
2,王五,3,语文
2,王五,3,数学
2,王五,3,英语
2,王五,3,政治
3,赵六,4,物理


### 六、对行或列进行删除

当我们想清理无效或无用的行或列

drop方法

**基本上，对DataFrame的操作方法，都是默认不改变原始DataFrame的，而是返回一个新的DataFrame**

**要么就得进行重新赋值，要么指定可选参数inplace=True**